# Quick tests on importing data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img
import os
import sys

module_path = os.path.abspath(os.path.join('..', 'time_serie_extraction'))
if module_path not in sys.path:
    sys.path.append(module_path)
from create_dataset import create_data_fixed_x, plot_image

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarn

### Loading data

In [2]:
save_file = r"/Users/fernandes/Documents/time_series_extraction/resources/dataset_fixed_x"

In [3]:
def import_data(folder_path, unit_shape=(300, 300, 4)):
    # Dealing with images
    image_paths = []
    for r, d, f in os.walk(folder_path):
        for file_path in f:
            image_paths.append(os.path.join(r, file_path))
    image_paths = [path for path in image_paths if ".png" in path]
    final_shape = [len(image_paths)] + list(unit_shape)
    output = np.ones(final_shape)
    for i, image_path in enumerate(image_paths):
        output[i] = img.imread(image_path)
    
    image_names = [path.split("/")[-1] for path in image_paths]
    
    # Sorting images by names
    df = pd.DataFrame(image_names)
    df["sort_trick"] = df[0].apply(lambda x: int(x.split(".")[0]))
    df = df.sort_values("sort_trick", ascending=True)
    output = output[df[0].index.values]
    
    # Dealing with targets
    target_path = os.path.join(save_file, "target.pkl")
    df_target = pd.read_pickle(target_path)
    return output, np.array(list(df_target.target))

In [4]:
x, y = import_data(save_file, unit_shape=(300, 300, 4))

In [5]:
# define the standalone discriminator model
def define_discriminator(in_shape=(300,300,4)):
    model = Sequential()

    model.add(Conv2D(32, (60,60), padding="same", input_shape=in_shape)) 
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(64, (20,20), padding="same"))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(128, (15,15), padding="valid"))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(LeakyReLU(alpha=0.2))

    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(8, activation="sigmoid"))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss="mse", optimizer=opt, metrics=["mse"]) 
    
    return model

In [6]:
model = define_discriminator(in_shape=(300,300,4))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 300, 300, 32)      460832    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 150, 150, 32)      0         
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 64)      819264    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 61, 61, 128)       1843328   
__________

In [ ]:
model.fit(x, y, epochs = 1, batch_size=4, verbose=1)

Epoch 1/1
196/400 [=============>................] - ETA: 39:14 - loss: 0.8004 - mean_squared_error: 0.8004